# Importing Required Packages

In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import collections
from collections import OrderedDict
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

## Read in raw data

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# node_1 data avx and non-avx(macro)
df_n1_avx = pd.read_csv("report_nodes1_icc_avx_oldgeom.csv")
df_n1_macro = pd.read_csv("report_nodes1_icc_macro_oldgeom.csv")

# node_2 data avx and non-avx(macro)
df_n2_avx = pd.read_csv("report_nodes2_icc_avx_oldgeom.csv")
df_n2_macro = pd.read_csv("report_nodes2_icc_macro_oldgeom.csv")

# 1-Node AVX Master Set
## Adding Columns to Current DataFrames

In [4]:
df_n1_avx["N_Nodes"] = 1
df_n1_avx['NPX'] = (df_n1_avx.GLB_X//df_n1_avx.LOC_X)
df_n1_avx['NPY'] = (df_n1_avx.GLB_Y//df_n1_avx.LOC_Y)
df_n1_avx['NPZ'] = (df_n1_avx.GLB_Z//df_n1_avx.LOC_Z)
df_n1_avx['NPT'] = (df_n1_avx.GLB_T//df_n1_avx.LOC_T)
df_n1_avx["Total_proc"] = df_n1_avx["omp_thrd"]* df_n1_avx.MPI_size
df_n1_avx.columns

Index(['omp_thrd', 'Gauge_group', 'MPI_size', 'GLB_T', 'GLB_X', 'GLB_Y',
       'GLB_Z', 'LOC_T', 'LOC_X', 'LOC_Y', 'LOC_Z', 'algb_kernel', 'geom_type',
       'RLXD', 'SITE_FLOP', 'SITE_BYTE', 'Dirc_Data_Mov', 'Dphi-Reps',
       'Dphi_KB', 'Dphi_msec', 'Dphi_GFLOPS', 'Dphi_BAND', 'Fused_Reps',
       'Fused_KB', 'Fused_msec', 'Fused_GFLOPS', 'Fused_BAND', 'Job_Output',
       'N_Nodes', 'NPX', 'NPY', 'NPZ', 'NPT', 'Total_proc'],
      dtype='object')

## Data Summary: 1-Node AVX Master

In [5]:
df_n1_avx_summary = df_n1_avx.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
df_n1_avx_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,2,13,2,1,1,1,26,61
1,2,14,2,1,1,1,28,61
2,2,15,2,1,1,1,30,61
3,2,16,2,1,1,1,32,61
4,4,7,4,1,1,1,28,29
5,4,8,4,1,1,1,32,29
6,6,5,6,1,1,1,30,18
7,8,4,8,1,1,1,32,13
8,10,3,10,1,1,1,30,9
9,14,2,14,1,1,1,28,6


## 1-Node AVX Cases

In [6]:
# Creating differnt data frames
dfn1avx_2mpi_16omp = df_n1_avx[(df_n1_avx.omp_thrd == 16) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 2) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]
dfn1avx_4mpi_8omp = df_n1_avx[(df_n1_avx.omp_thrd == 8) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 4) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]
dfn1avx_8mpi_4omp = df_n1_avx[(df_n1_avx.omp_thrd == 4) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 8) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z ==1)]
dfn1avx_16mpi_2omp = df_n1_avx[(df_n1_avx.omp_thrd == 2) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 16) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]
dfn1avx_32mpi_1omp = df_n1_avx[(df_n1_avx.omp_thrd == 1) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 32) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]
dfn1avx_2mpi_15omp = df_n1_avx[(df_n1_avx.omp_thrd == 15) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 2) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]
dfn1avx_2mpi_14omp = df_n1_avx[(df_n1_avx.omp_thrd == 14) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 2) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]
dfn1avx_2mpi_13omp = df_n1_avx[(df_n1_avx.omp_thrd == 13) & (df_n1_avx.GLB_T/df_n1_avx.LOC_T == 2) & (df_n1_avx.GLB_X/df_n1_avx.LOC_X == 1) & (df_n1_avx.GLB_Y/df_n1_avx.LOC_Y ==1)& (df_n1_avx.GLB_Z/df_n1_avx.LOC_Z == 1)]

### 1-Node FLOPs Measuement

In [7]:

fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_16omp.Dirc_Data_Mov*(dfn1avx_2mpi_16omp.MPI_size)), 
                         y=(dfn1avx_2mpi_16omp.Fused_GFLOPS*(dfn1avx_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_16omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_16omp.Dirc_Data_Mov*(dfn1avx_2mpi_16omp.MPI_size)), 
                         y=(dfn1avx_2mpi_16omp.Dphi_GFLOPS*(dfn1avx_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_16omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))



fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_15omp.Dirc_Data_Mov*(dfn1avx_2mpi_15omp.MPI_size)), 
                         y=(dfn1avx_2mpi_15omp.Fused_GFLOPS*(dfn1avx_2mpi_15omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_15omp',
                         line=dict(color='gold',width=2),
                         marker=dict(symbol='triangle-up',size=5, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_15omp.Dirc_Data_Mov*(dfn1avx_2mpi_15omp.MPI_size)), 
                         y=(dfn1avx_2mpi_15omp.Dphi_GFLOPS*(dfn1avx_2mpi_15omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_15omp',
                         line=dict(color='dimgray',width=2),
                         marker=dict(symbol='triangle-up',size=5, line=dict(color='black',width=1))))



fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_14omp.Dirc_Data_Mov*(dfn1avx_2mpi_14omp.MPI_size)), 
                         y=(dfn1avx_2mpi_14omp.Fused_GFLOPS*(dfn1avx_2mpi_14omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_14omp',
                         line=dict(color='black',width=2),
                         marker=dict(symbol='diamond-x',size=4, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_14omp.Dirc_Data_Mov*(dfn1avx_2mpi_14omp.MPI_size)), 
                         y=(dfn1avx_2mpi_14omp.Dphi_GFLOPS*(dfn1avx_2mpi_14omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_14omp',
                         line=dict(color='dimgray',width=2),
                         marker=dict(symbol='diamond-x',size=4, line=dict(color='yellow',width=1))))



fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_13omp.Dirc_Data_Mov*(dfn1avx_2mpi_13omp.MPI_size)), 
                         y=(dfn1avx_2mpi_13omp.Fused_GFLOPS*(dfn1avx_2mpi_13omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_13omp',
                         line=dict(color='salmon',width=2),
                         marker=dict(symbol='circle-cross',size=4, line=dict(color='green',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_13omp.Dirc_Data_Mov*(dfn1avx_2mpi_13omp.MPI_size)), 
                         y=(dfn1avx_2mpi_13omp.Dphi_GFLOPS*(dfn1avx_2mpi_13omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_13omp',
                         line=dict(color='skyblue',width=2),
                         marker=dict(symbol='circle-cross',size=4, line=dict(color='green',width=1))))



fig.add_trace(go.Scatter(x=(dfn1avx_4mpi_8omp.Dirc_Data_Mov*(dfn1avx_4mpi_8omp.MPI_size)), 
                        y=(dfn1avx_4mpi_8omp.Fused_GFLOPS*(dfn1avx_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_8omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1avx_4mpi_8omp.Dirc_Data_Mov*(dfn1avx_4mpi_8omp.MPI_size)), 
                         y=(dfn1avx_4mpi_8omp.Dphi_GFLOPS*(dfn1avx_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_8omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



fig.add_trace(go.Scatter(x=(dfn1avx_8mpi_4omp.Dirc_Data_Mov*(dfn1avx_8mpi_4omp.MPI_size)), 
                         y=(dfn1avx_8mpi_4omp.Fused_GFLOPS*(dfn1avx_8mpi_4omp.MPI_size)),
                         mode='lines+markers', 
                         name='Fused_8mpi_4omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_8mpi_4omp.Dirc_Data_Mov*(dfn1avx_8mpi_4omp.MPI_size)), 
                         y=(dfn1avx_8mpi_4omp.Dphi_GFLOPS*(dfn1avx_8mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_8mpi_4omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn1avx_16mpi_2omp.Dirc_Data_Mov*(dfn1avx_16mpi_2omp.MPI_size)), 
                         y=(dfn1avx_16mpi_2omp.Fused_GFLOPS*(dfn1avx_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1avx_16mpi_2omp.Dirc_Data_Mov*(dfn1avx_16mpi_2omp.MPI_size)), 
                         y=(dfn1avx_16mpi_2omp.Dphi_GFLOPS*(dfn1avx_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1avx_32mpi_1omp.Dirc_Data_Mov*(dfn1avx_32mpi_1omp.MPI_size)), 
                         y=(dfn1avx_32mpi_1omp.Fused_GFLOPS*(dfn1avx_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_1omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_32mpi_1omp.Dirc_Data_Mov*(dfn1avx_32mpi_1omp.MPI_size)), 
                         y=(dfn1avx_32mpi_1omp.Dphi_GFLOPS*(dfn1avx_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_1omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))


# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"1_Node_AVX" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

In [8]:
# dfn1avx_2mpi_16omp
# dfn1avx_4mpi_8omp
# dfn1avx_8mpi_4omp
# dfn1avx_16mpi_2omp
dfn1avx_32mpi_1omp

,omp_thrd,Gauge_group,MPI_size,GLB_T,GLB_X,GLB_Y,GLB_Z,LOC_T,LOC_X,LOC_Y,LOC_Z,algb_kernel,geom_type,RLXD,SITE_FLOP,SITE_BYTE,Dirc_Data_Mov,Dphi-Reps,Dphi_KB,Dphi_msec,Dphi_GFLOPS,Dphi_BAND,Fused_Reps,Fused_KB,Fused_msec,Fused_GFLOPS,Fused_BAND,Job_Output,N_Nodes,NPX,NPY,NPZ,NPT,Total_proc
356,1,3,32,128,8,8,8,4,8,8,8,AVX2,OLD,12345,1320,3072,6291456,2639,6144.0,2205.194092,3.23517,7.52911,2639,6144.0,2271.303955,3.141,7.30996,job.na.out,1,1,1,1,32,32


### 1-Node Bandwidth Measurement

In [9]:

fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_16omp.Dirc_Data_Mov*(dfn1avx_2mpi_16omp.MPI_size)), 
                         y=(dfn1avx_2mpi_16omp.Fused_BAND*(dfn1avx_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_16omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_16omp.Dirc_Data_Mov*(dfn1avx_2mpi_16omp.MPI_size)), 
                         y=(dfn1avx_2mpi_16omp.Dphi_BAND*(dfn1avx_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_16omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))





fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_15omp.Dirc_Data_Mov*(dfn1avx_2mpi_15omp.MPI_size)), 
                         y=(dfn1avx_2mpi_15omp.Fused_BAND*(dfn1avx_2mpi_15omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_15omp',
                         line=dict(color='gold',width=2),
                         marker=dict(symbol='triangle-up',size=5, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_15omp.Dirc_Data_Mov*(dfn1avx_2mpi_15omp.MPI_size)), 
                         y=(dfn1avx_2mpi_15omp.Dphi_BAND*(dfn1avx_2mpi_15omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_15omp',
                         line=dict(color='dimgray',width=2),
                         marker=dict(symbol='triangle-up',size=5, line=dict(color='black',width=1))))



fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_14omp.Dirc_Data_Mov*(dfn1avx_2mpi_14omp.MPI_size)), 
                         y=(dfn1avx_2mpi_14omp.Fused_BAND*(dfn1avx_2mpi_14omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_14omp',
                         line=dict(color='tomato',width=2),
                         marker=dict(symbol='diamond-x',size=4, line=dict(color='lawngreen',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_14omp.Dirc_Data_Mov*(dfn1avx_2mpi_14omp.MPI_size)), 
                         y=(dfn1avx_2mpi_14omp.Dphi_BAND*(dfn1avx_2mpi_14omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_14omp',
                         line=dict(color='purple',width=2),
                         marker=dict(symbol='diamond-x',size=4, line=dict(color='lawngreen',width=1))))



fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_13omp.Dirc_Data_Mov*(dfn1avx_2mpi_13omp.MPI_size)), 
                         y=(dfn1avx_2mpi_13omp.Fused_BAND*(dfn1avx_2mpi_13omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_13omp',
                         line=dict(color='salmon',width=2),
                         marker=dict(symbol='circle-cross',size=4, line=dict(color='green',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_13omp.Dirc_Data_Mov*(dfn1avx_2mpi_13omp.MPI_size)), 
                         y=(dfn1avx_2mpi_13omp.Dphi_BAND*(dfn1avx_2mpi_13omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_13omp',
                         line=dict(color='skyblue',width=2),
                         marker=dict(symbol='circle-cross',size=4, line=dict(color='green',width=1))))




fig.add_trace(go.Scatter(x=(dfn1avx_4mpi_8omp.Dirc_Data_Mov*(dfn1avx_4mpi_8omp.MPI_size)), 
                        y=(dfn1avx_4mpi_8omp.Fused_BAND*(dfn1avx_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_8omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1avx_4mpi_8omp.Dirc_Data_Mov*(dfn1avx_4mpi_8omp.MPI_size)), 
                         y=(dfn1avx_4mpi_8omp.Dphi_BAND*(dfn1avx_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_8omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn1avx_8mpi_4omp.Dirc_Data_Mov*(dfn1avx_8mpi_4omp.MPI_size)), 
                         y=(dfn1avx_8mpi_4omp.Fused_BAND*(dfn1avx_8mpi_4omp.MPI_size)),
                         mode='lines+markers', 
                         name='Fused_8mpi_4omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_8mpi_4omp.Dirc_Data_Mov*(dfn1avx_8mpi_4omp.MPI_size)), 
                         y=(dfn1avx_8mpi_4omp.Dphi_BAND*(dfn1avx_8mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_8mpi_4omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn1avx_16mpi_2omp.Dirc_Data_Mov*(dfn1avx_16mpi_2omp.MPI_size)), 
                         y=(dfn1avx_16mpi_2omp.Fused_BAND*(dfn1avx_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1avx_16mpi_2omp.Dirc_Data_Mov*(dfn1avx_16mpi_2omp.MPI_size)), 
                         y=(dfn1avx_16mpi_2omp.Dphi_BAND*(dfn1avx_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1avx_32mpi_1omp.Dirc_Data_Mov*(dfn1avx_32mpi_1omp.MPI_size)), 
                         y=(dfn1avx_32mpi_1omp.Fused_BAND*(dfn1avx_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_1omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_32mpi_1omp.Dirc_Data_Mov*(dfn1avx_32mpi_1omp.MPI_size)), 
                         y=(dfn1avx_32mpi_1omp.Dphi_BAND*(dfn1avx_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_1omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))


# Adding vertical and horizontal lines 
fig.add_vline(x=80e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=153, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='80e6'
    , y=280
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=155
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"1_Node_AVX" Bandwidth Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside")) 

In [10]:
# dfn1avx_2mpi_16omp
# dfn1avx_4mpi_8omp
# dfn1avx_8mpi_4omp
# dfn1avx_16mpi_2omp
# dfn1avx_32mpi_1omp

# 1-Node Macro Master Set
## Adding Columns to Current DataFrames

In [11]:
df_n1_macro["N_Nodes"] = 1
df_n1_macro['NPX'] = (df_n1_macro.GLB_X//df_n1_macro.LOC_X)
df_n1_macro['NPY'] = (df_n1_macro.GLB_Y//df_n1_macro.LOC_Y)
df_n1_macro['NPZ'] = (df_n1_macro.GLB_Z//df_n1_macro.LOC_Z)
df_n1_macro['NPT'] = (df_n1_macro.GLB_T//df_n1_macro.LOC_T)
df_n1_macro["Total_proc"] = df_n1_macro["omp_thrd"]* df_n1_macro.MPI_size
df_n1_macro.columns

Index(['omp_thrd', 'Gauge_group', 'MPI_size', 'GLB_T', 'GLB_X', 'GLB_Y',
       'GLB_Z', 'LOC_T', 'LOC_X', 'LOC_Y', 'LOC_Z', 'algb_kernel', 'geom_type',
       'RLXD', 'SITE_FLOP', 'SITE_BYTE', 'Dirc_Data_Mov', 'Dphi-Reps',
       'Dphi_KB', 'Dphi_msec', 'Dphi_GFLOPS', 'Dphi_BAND', 'Fused_Reps',
       'Fused_KB', 'Fused_msec', 'Fused_GFLOPS', 'Fused_BAND', 'Job_Output',
       'N_Nodes', 'NPX', 'NPY', 'NPZ', 'NPT', 'Total_proc'],
      dtype='object')

## Data Summary: 1-Node Macro Master

In [12]:
df_n1_macro_summary = df_n1_macro.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
df_n1_macro_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,2,13,2,1,1,1,26,61
1,2,14,2,1,1,1,28,61
2,2,15,2,1,1,1,30,61
3,2,16,2,1,1,1,32,61
4,4,7,4,1,1,1,28,29
5,4,8,4,1,1,1,32,29
6,6,5,6,1,1,1,30,18
7,8,4,8,1,1,1,32,13
8,10,3,10,1,1,1,30,9
9,14,2,14,1,1,1,28,6


## 1-Node Macro Cases

In [13]:
# Creating differnt data frames
dfn1macro_2mpi_16omp = df_n1_macro[(df_n1_macro.omp_thrd == 16) & (df_n1_macro.GLB_T/df_n1_macro.LOC_T == 2) & (df_n1_macro.GLB_X/df_n1_macro.LOC_X == 1) & (df_n1_macro.GLB_Y/df_n1_macro.LOC_Y ==1)& (df_n1_macro.GLB_Z/df_n1_macro.LOC_Z == 1)]
dfn1macro_4mpi_8omp = df_n1_macro[(df_n1_macro.omp_thrd == 8) & (df_n1_macro.GLB_T/df_n1_macro.LOC_T == 4) & (df_n1_macro.GLB_X/df_n1_macro.LOC_X == 1) & (df_n1_macro.GLB_Y/df_n1_macro.LOC_Y ==1)& (df_n1_macro.GLB_Z/df_n1_macro.LOC_Z == 1)]
dfn1macro_8mpi_4omp = df_n1_macro[(df_n1_macro.omp_thrd == 4) & (df_n1_macro.GLB_T/df_n1_macro.LOC_T == 8) & (df_n1_macro.GLB_X/df_n1_macro.LOC_X == 1) & (df_n1_macro.GLB_Y/df_n1_macro.LOC_Y ==1)& (df_n1_macro.GLB_Z/df_n1_macro.LOC_Z ==1)]
dfn1macro_16mpi_2omp = df_n1_macro[(df_n1_macro.omp_thrd == 2) & (df_n1_macro.GLB_T/df_n1_macro.LOC_T == 16) & (df_n1_macro.GLB_X/df_n1_macro.LOC_X == 1) & (df_n1_macro.GLB_Y/df_n1_macro.LOC_Y ==1)& (df_n1_macro.GLB_Z/df_n1_macro.LOC_Z == 1)]
dfn1macro_32mpi_1omp = df_n1_macro[(df_n1_macro.omp_thrd == 1) & (df_n1_macro.GLB_T/df_n1_macro.LOC_T == 32) & (df_n1_macro.GLB_X/df_n1_macro.LOC_X == 1) & (df_n1_macro.GLB_Y/df_n1_macro.LOC_Y ==1)& (df_n1_macro.GLB_Z/df_n1_macro.LOC_Z == 1)]

### 1-Node FLOPs Measuement

In [14]:

fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1macro_2mpi_16omp.Dirc_Data_Mov*(dfn1macro_2mpi_16omp.MPI_size)), 
                         y=(dfn1macro_2mpi_16omp.Fused_GFLOPS*(dfn1macro_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_16omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_2mpi_16omp.Dirc_Data_Mov*(dfn1macro_2mpi_16omp.MPI_size)), 
                         y=(dfn1macro_2mpi_16omp.Dphi_GFLOPS*(dfn1macro_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_16omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn1macro_4mpi_8omp.Dirc_Data_Mov*(dfn1macro_4mpi_8omp.MPI_size)), 
                        y=(dfn1macro_4mpi_8omp.Fused_GFLOPS*(dfn1macro_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_8omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1macro_4mpi_8omp.Dirc_Data_Mov*(dfn1macro_4mpi_8omp.MPI_size)), 
                         y=(dfn1macro_4mpi_8omp.Dphi_GFLOPS*(dfn1macro_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_8omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn1macro_8mpi_4omp.Dirc_Data_Mov*(dfn1macro_8mpi_4omp.MPI_size)), 
                         y=(dfn1macro_8mpi_4omp.Fused_GFLOPS*(dfn1macro_8mpi_4omp.MPI_size)),
                         mode='lines+markers', 
                         name='Fused_8mp_4omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_8mpi_4omp.Dirc_Data_Mov*(dfn1macro_8mpi_4omp.MPI_size)), 
                         y=(dfn1macro_8mpi_4omp.Dphi_GFLOPS*(dfn1macro_8mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_8mp_4omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn1macro_16mpi_2omp.Dirc_Data_Mov*(dfn1macro_16mpi_2omp.MPI_size)), 
                         y=(dfn1macro_16mpi_2omp.Fused_GFLOPS*(dfn1macro_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1macro_16mpi_2omp.Dirc_Data_Mov*(dfn1macro_16mpi_2omp.MPI_size)), 
                         y=(dfn1macro_16mpi_2omp.Dphi_GFLOPS*(dfn1macro_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1macro_32mpi_1omp.Dirc_Data_Mov*(dfn1macro_32mpi_1omp.MPI_size)), 
                         y=(dfn1macro_32mpi_1omp.Fused_GFLOPS*(dfn1macro_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_1omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_32mpi_1omp.Dirc_Data_Mov*(dfn1macro_32mpi_1omp.MPI_size)), 
                         y=(dfn1macro_32mpi_1omp.Dphi_GFLOPS*(dfn1macro_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_1omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))


# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"1_Node_Macro" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

In [15]:
# dfn1macro_2mpi_16omp
# dfn1macro_4mpi_8omp
# dfn1macro_8mpi_4omp
# dfn1macro_16mpi_2omp
# dfn1macro_32mpi_1omp

### 1-Node Bandwidht Measuement

In [16]:

fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1macro_2mpi_16omp.Dirc_Data_Mov*(dfn1macro_2mpi_16omp.MPI_size)), 
                         y=(dfn1macro_2mpi_16omp.Fused_BAND*(dfn1macro_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_16omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_2mpi_16omp.Dirc_Data_Mov*(dfn1macro_2mpi_16omp.MPI_size)), 
                         y=(dfn1macro_2mpi_16omp.Dphi_BAND*(dfn1macro_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_16omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn1macro_4mpi_8omp.Dirc_Data_Mov*(dfn1macro_4mpi_8omp.MPI_size)), 
                        y=(dfn1macro_4mpi_8omp.Fused_BAND*(dfn1macro_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_8omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1macro_4mpi_8omp.Dirc_Data_Mov*(dfn1macro_4mpi_8omp.MPI_size)), 
                         y=(dfn1macro_4mpi_8omp.Dphi_BAND*(dfn1macro_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_8omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn1macro_8mpi_4omp.Dirc_Data_Mov*(dfn1macro_8mpi_4omp.MPI_size)), 
                         y=(dfn1macro_8mpi_4omp.Fused_BAND*(dfn1macro_8mpi_4omp.MPI_size)),
                         mode='lines+markers', 
                         name='Fused_8mp_4omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_8mpi_4omp.Dirc_Data_Mov*(dfn1macro_8mpi_4omp.MPI_size)), 
                         y=(dfn1macro_8mpi_4omp.Dphi_BAND*(dfn1macro_8mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_8mp_4omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn1macro_16mpi_2omp.Dirc_Data_Mov*(dfn1macro_16mpi_2omp.MPI_size)), 
                         y=(dfn1macro_16mpi_2omp.Fused_BAND*(dfn1macro_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1macro_16mpi_2omp.Dirc_Data_Mov*(dfn1macro_16mpi_2omp.MPI_size)), 
                         y=(dfn1macro_16mpi_2omp.Dphi_BAND*(dfn1macro_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1macro_32mpi_1omp.Dirc_Data_Mov*(dfn1macro_32mpi_1omp.MPI_size)), 
                         y=(dfn1macro_32mpi_1omp.Fused_BAND*(dfn1macro_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_1omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_32mpi_1omp.Dirc_Data_Mov*(dfn1macro_32mpi_1omp.MPI_size)), 
                         y=(dfn1macro_32mpi_1omp.Dphi_BAND*(dfn1macro_32mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_1omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))


# Adding vertical and horizontal lines 
fig.add_vline(x=80e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=153, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='80e6'
    , y=310
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=155
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"1_Node_Macro" Bandwidth Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside")) 

In [17]:
# dfn1macro_2mpi_16omp
# dfn1macro_4mpi_8omp
# dfn1macro_8mpi_4omp
# dfn1macro_16mpi_2omp
# dfn1macro_32mpi_1omp


## Node-1 case 1: avx vs macro 2 mpi 16 omp

In [18]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_16omp.Dirc_Data_Mov*(dfn1avx_2mpi_16omp.MPI_size)), 
                         y=(dfn1avx_2mpi_16omp.Fused_GFLOPS*(dfn1avx_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx-Fused_2mpi_16omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_2mpi_16omp.Dirc_Data_Mov*(dfn1avx_2mpi_16omp.MPI_size)), 
                         y=(dfn1avx_2mpi_16omp.Dphi_GFLOPS*(dfn1avx_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx-Dphi_2mpi_16omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))



fig.add_trace(go.Scatter(x=(dfn1macro_2mpi_16omp.Dirc_Data_Mov*(dfn1macro_2mpi_16omp.MPI_size)), 
                         y=(dfn1macro_2mpi_16omp.Fused_GFLOPS*(dfn1macro_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro-Fused_2mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_2mpi_16omp.Dirc_Data_Mov*(dfn1macro_2mpi_16omp.MPI_size)), 
                         y=(dfn1macro_2mpi_16omp.Dphi_GFLOPS*(dfn1macro_2mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro-Dphi_2mpi_16omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"1_Node_Macro" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))


## Node-1 case 2: avx vs macro 4mpi 8omp 

In [19]:

fig = go.Figure() 
fig.add_trace(go.Scatter(x=(dfn1avx_4mpi_8omp.Dirc_Data_Mov*(dfn1avx_4mpi_8omp.MPI_size)), 
                        y=(dfn1avx_4mpi_8omp.Fused_GFLOPS*(dfn1avx_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx-Fused_4mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1avx_4mpi_8omp.Dirc_Data_Mov*(dfn1avx_4mpi_8omp.MPI_size)), 
                         y=(dfn1avx_4mpi_8omp.Dphi_GFLOPS*(dfn1avx_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx-Dphi_4mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn1macro_4mpi_8omp.Dirc_Data_Mov*(dfn1macro_4mpi_8omp.MPI_size)), 
                        y=(dfn1macro_4mpi_8omp.Fused_GFLOPS*(dfn1macro_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro-Fused_4mpi_8omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1macro_4mpi_8omp.Dirc_Data_Mov*(dfn1macro_4mpi_8omp.MPI_size)), 
                         y=(dfn1macro_4mpi_8omp.Dphi_GFLOPS*(dfn1macro_4mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro-Dphi_4mpi_8omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"1_Node_Macro" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))




## Node-1 case 3: avx-macro 8 mpi 4 omp 

In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=(dfn1avx_8mpi_4omp.Dirc_Data_Mov*(dfn1avx_8mpi_4omp.MPI_size)), 
                         y=(dfn1avx_8mpi_4omp.Fused_GFLOPS*(dfn1avx_8mpi_4omp.MPI_size)),
                         mode='lines+markers', 
                         name='avx-Fused_8mpi_4omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1avx_8mpi_4omp.Dirc_Data_Mov*(dfn1avx_8mpi_4omp.MPI_size)), 
                         y=(dfn1avx_8mpi_4omp.Dphi_GFLOPS*(dfn1avx_8mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx-Dphi_8mpi_4omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn1macro_8mpi_4omp.Dirc_Data_Mov*(dfn1macro_8mpi_4omp.MPI_size)), 
                         y=(dfn1macro_8mpi_4omp.Fused_GFLOPS*(dfn1macro_8mpi_4omp.MPI_size)),
                         mode='lines+markers', 
                         name='macro-Fused_8mp_4omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn1macro_8mpi_4omp.Dirc_Data_Mov*(dfn1macro_8mpi_4omp.MPI_size)), 
                         y=(dfn1macro_8mpi_4omp.Dphi_GFLOPS*(dfn1macro_8mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro-Dphi_8mp_4omp',
                         line=dict(color='orange',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))



# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"1_Node_Macro" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))


## Node-1 Case 4: avx vs macro 16 mpi 2 omp

In [21]:

fig = go.Figure()


fig.add_trace(go.Scatter(x=(dfn1avx_16mpi_2omp.Dirc_Data_Mov*(dfn1avx_16mpi_2omp.MPI_size)), 
                         y=(dfn1avx_16mpi_2omp.Fused_GFLOPS*(dfn1avx_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx_Fused_16mpi_2omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1avx_16mpi_2omp.Dirc_Data_Mov*(dfn1avx_16mpi_2omp.MPI_size)), 
                         y=(dfn1avx_16mpi_2omp.Dphi_GFLOPS*(dfn1avx_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='avx_Dphi_16mpi_2omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1macro_16mpi_2omp.Dirc_Data_Mov*(dfn1macro_16mpi_2omp.MPI_size)), 
                         y=(dfn1macro_16mpi_2omp.Fused_GFLOPS*(dfn1macro_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro_Fused_16mpi_2omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn1macro_16mpi_2omp.Dirc_Data_Mov*(dfn1macro_16mpi_2omp.MPI_size)), 
                         y=(dfn1macro_16mpi_2omp.Dphi_GFLOPS*(dfn1macro_16mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='macro_Dphi_16mpi_2omp',
                         line=dict(color='orange',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"1_Node_Macro" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))



## 2-Node AVX Master Set

## Adding Columns to Current DataFrames

In [22]:
df_n2_avx["N_Nodes"] = 2
df_n2_avx['NPX'] = (df_n2_avx.GLB_X//df_n2_avx.LOC_X)
df_n2_avx['NPY'] = (df_n2_avx.GLB_Y//df_n2_avx.LOC_Y)
df_n2_avx['NPZ'] = (df_n2_avx.GLB_Z//df_n2_avx.LOC_Z)
df_n2_avx['NPT'] = (df_n2_avx.GLB_T//df_n2_avx.LOC_T)
df_n2_avx["Total_proc"] = df_n2_avx["omp_thrd"]* df_n2_avx.MPI_size
df_n2_avx.columns

Index(['omp_thrd', 'Gauge_group', 'MPI_size', 'GLB_T', 'GLB_X', 'GLB_Y',
       'GLB_Z', 'LOC_T', 'LOC_X', 'LOC_Y', 'LOC_Z', 'algb_kernel', 'geom_type',
       'RLXD', 'SITE_FLOP', 'SITE_BYTE', 'Dirc_Data_Mov', 'Dphi-Reps',
       'Dphi_KB', 'Dphi_msec', 'Dphi_GFLOPS', 'Dphi_BAND', 'Fused_Reps',
       'Fused_KB', 'Fused_msec', 'Fused_GFLOPS', 'Fused_BAND', 'Job_Output',
       'N_Nodes', 'NPX', 'NPY', 'NPZ', 'NPT', 'Total_proc'],
      dtype='object')

In [23]:
# Clean Data: Drop the rows have empty cells
df_n2_avx=df_n2_avx.dropna()

## Data Summary: 2-Node AVX Master

In [24]:
df_n2_avx_summary = df_n2_avx.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
df_n2_avx_summary 

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,2,26,2,1,1,1,52,125
1,2,27,2,1,1,1,54,125
2,2,28,2,1,1,1,56,125
3,2,29,2,1,1,1,58,125
4,2,30,2,1,1,1,60,125
5,2,31,2,1,1,1,62,125
6,2,32,2,1,1,1,64,125
7,4,13,4,1,1,1,52,61
8,4,14,4,1,1,1,56,61
9,4,15,4,1,1,1,60,61


## 2-Node AVX cases

## 2-Node AVX FLOPs Measuement

In [25]:
# Creating differnt data frames
dfn2avx_2mpi_32omp = df_n2_avx[(df_n2_avx.omp_thrd == 32) & (df_n2_avx.GLB_T/df_n2_avx.LOC_T == 2) & (df_n2_avx.GLB_X/df_n2_avx.LOC_X == 1) & (df_n2_avx.GLB_Y/df_n2_avx.LOC_Y ==1)& (df_n2_avx.GLB_Z/df_n2_avx.LOC_Z == 1)]
dfn2avx_4mpi_16omp = df_n2_avx[(df_n2_avx.omp_thrd == 16) & (df_n2_avx.GLB_T/df_n2_avx.LOC_T == 4) & (df_n2_avx.GLB_X/df_n2_avx.LOC_X == 1) & (df_n2_avx.GLB_Y/df_n2_avx.LOC_Y ==1)& (df_n2_avx.GLB_Z/df_n2_avx.LOC_Z == 1)]
dfn2avx_8mpi_8omp = df_n2_avx[(df_n2_avx.omp_thrd == 8) & (df_n2_avx.GLB_T/df_n2_avx.LOC_T == 8) & (df_n2_avx.GLB_X/df_n2_avx.LOC_X == 1) & (df_n2_avx.GLB_Y/df_n2_avx.LOC_Y ==1)& (df_n2_avx.GLB_Z/df_n2_avx.LOC_Z ==1)]
dfn2avx_16mpi_4omp = df_n2_avx[(df_n2_avx.omp_thrd == 4) & (df_n2_avx.GLB_T/df_n2_avx.LOC_T == 16) & (df_n2_avx.GLB_X/df_n2_avx.LOC_X == 1) & (df_n2_avx.GLB_Y/df_n2_avx.LOC_Y ==1)& (df_n2_avx.GLB_Z/df_n2_avx.LOC_Z == 1)]
dfn2avx_32mpi_2omp = df_n2_avx[(df_n2_avx.omp_thrd == 2) & (df_n2_avx.GLB_T/df_n2_avx.LOC_T == 32) & (df_n2_avx.GLB_X/df_n2_avx.LOC_X == 1) & (df_n2_avx.GLB_Y/df_n2_avx.LOC_Y ==1)& (df_n2_avx.GLB_Z/df_n2_avx.LOC_Z == 1)]
dfn2avx_64mpi_1omp = df_n2_avx[(df_n2_avx.omp_thrd == 1) & (df_n2_avx.GLB_T/df_n2_avx.LOC_T == 64) & (df_n2_avx.GLB_X/df_n2_avx.LOC_X == 1) & (df_n2_avx.GLB_Y/df_n2_avx.LOC_Y ==1)& (df_n2_avx.GLB_Z/df_n2_avx.LOC_Z == 1)]


In [26]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2avx_2mpi_32omp.Dirc_Data_Mov*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2avx_2mpi_32omp.Fused_GFLOPS*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_32omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn2avx_2mpi_32omp.Dirc_Data_Mov*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2avx_2mpi_32omp.Dphi_GFLOPS*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_32omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2avx_4mpi_16omp.Dirc_Data_Mov*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                        y=(dfn2avx_4mpi_16omp.Fused_GFLOPS*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_16omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2avx_4mpi_16omp.Dirc_Data_Mov*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                         y=(dfn2avx_4mpi_16omp.Dphi_GFLOPS*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_16omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn2avx_8mpi_8omp.Dirc_Data_Mov*(dfn2avx_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2avx_8mpi_8omp.Fused_GFLOPS*(dfn2avx_8mpi_8omp.MPI_size/2)),
                         mode='lines+markers', 
                         name='Fused_8mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn2avx_8mpi_8omp.Dirc_Data_Mov*(dfn2avx_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2avx_8mpi_8omp.Dphi_GFLOPS*(dfn2avx_8mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_8mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn2avx_16mpi_4omp.Dirc_Data_Mov*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2avx_16mpi_4omp.Fused_GFLOPS*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_4omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2avx_16mpi_4omp.Dirc_Data_Mov*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2avx_16mpi_4omp.Dphi_GFLOPS*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_4omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn2avx_32mpi_2omp.Dirc_Data_Mov*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2avx_32mpi_2omp.Fused_GFLOPS*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_2omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn2avx_32mpi_2omp.Dirc_Data_Mov*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2avx_32mpi_2omp.Dphi_GFLOPS*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_2omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))



fig.add_trace(go.Scatter(x=(dfn2avx_64mpi_1omp.Dirc_Data_Mov*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2avx_64mpi_1omp.Fused_GFLOPS*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_64mpi_1omp',
                         line=dict(color='olive',width=3)))
fig.add_trace(go.Scatter(x=(dfn2avx_64mpi_1omp.Dirc_Data_Mov*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2avx_64mpi_1omp.Dphi_GFLOPS*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_64mpi_1omp',
                         line=dict(color='lightskyblue',width=3)))


# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"2_Node_AVX" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

In [27]:
# dfn2avx_64mpi_1omp
# dfn2avx_4mpi_16omp
# dfn2avx_8mpi_8omp
# dfn2avx_16mpi_4omp
# dfn2avx_32mpi_2omp
# dfn2avx_64mpi_1omp

## 2-Node AVX Bandwidth Measuement

In [28]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2avx_2mpi_32omp.Dirc_Data_Mov*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2avx_2mpi_32omp.Fused_BAND*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_32omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn2avx_2mpi_32omp.Dirc_Data_Mov*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2avx_2mpi_32omp.Dphi_BAND*(dfn2avx_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_32omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2avx_4mpi_16omp.Dirc_Data_Mov*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                        y=(dfn2avx_4mpi_16omp.Fused_BAND*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_16omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2avx_4mpi_16omp.Dirc_Data_Mov*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                         y=(dfn2avx_4mpi_16omp.Dphi_BAND*(dfn2avx_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_16omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn2avx_8mpi_8omp.Dirc_Data_Mov*(dfn2avx_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2avx_8mpi_8omp.Fused_BAND*(dfn2avx_8mpi_8omp.MPI_size/2)),
                         mode='lines+markers', 
                         name='Fused_8mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn2avx_8mpi_8omp.Dirc_Data_Mov*(dfn2avx_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2avx_8mpi_8omp.Dphi_BAND*(dfn2avx_8mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_8mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn2avx_16mpi_4omp.Dirc_Data_Mov*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2avx_16mpi_4omp.Fused_BAND*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_4omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2avx_16mpi_4omp.Dirc_Data_Mov*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2avx_16mpi_4omp.Dphi_BAND*(dfn2avx_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_4omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn2avx_32mpi_2omp.Dirc_Data_Mov*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2avx_32mpi_2omp.Fused_BAND*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_2omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn2avx_32mpi_2omp.Dirc_Data_Mov*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2avx_32mpi_2omp.Dphi_BAND*(dfn2avx_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_2omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))



fig.add_trace(go.Scatter(x=(dfn2avx_64mpi_1omp.Dirc_Data_Mov*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2avx_64mpi_1omp.Fused_BAND*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_64mpi_1omp',
                         line=dict(color='olive',width=3)))
fig.add_trace(go.Scatter(x=(dfn2avx_64mpi_1omp.Dirc_Data_Mov*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2avx_64mpi_1omp.Dphi_BAND*(dfn2avx_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_64mpi_1omp',
                         line=dict(color='lightskyblue',width=3)))


# Adding vertical and horizontal lines 
fig.add_vline(x=80e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=153, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='80e6'
    , y=280
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=155
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"2_Node_AVX" Bandwidth Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside")) 


In [29]:
# dfn2avx_64mpi_1omp
# dfn2avx_4mpi_16omp
# dfn2avx_8mpi_8omp
# dfn2avx_16mpi_4omp
# dfn2avx_32mpi_2omp
# dfn2avx_64mpi_1omp

## 2-Node Macro Master Set

## Adding Columns to Current DataFrames

In [30]:
df_n2_macro["N_Nodes"] = 2
df_n2_macro['NPX'] = (df_n2_macro.GLB_X//df_n2_macro.LOC_X)
df_n2_macro['NPY'] = (df_n2_macro.GLB_Y//df_n2_macro.LOC_Y)
df_n2_macro['NPZ'] = (df_n2_macro.GLB_Z//df_n2_macro.LOC_Z)
df_n2_macro['NPT'] = (df_n2_macro.GLB_T//df_n2_macro.LOC_T)
df_n2_macro["Total_proc"] = df_n2_macro["omp_thrd"]* df_n2_macro.MPI_size
df_n2_macro.columns

Index(['omp_thrd', 'Gauge_group', 'MPI_size', 'GLB_T', 'GLB_X', 'GLB_Y',
       'GLB_Z', 'LOC_T', 'LOC_X', 'LOC_Y', 'LOC_Z', 'algb_kernel', 'geom_type',
       'RLXD', 'SITE_FLOP', 'SITE_BYTE', 'Dirc_Data_Mov', 'Dphi-Reps',
       'Dphi_KB', 'Dphi_msec', 'Dphi_GFLOPS', 'Dphi_BAND', 'Fused_Reps',
       'Fused_KB', 'Fused_msec', 'Fused_GFLOPS', 'Fused_BAND', 'Job_Output',
       'N_Nodes', 'NPX', 'NPY', 'NPZ', 'NPT', 'Total_proc'],
      dtype='object')

In [31]:
# Drop the rows have empty cells
df_n2_macro=df_n2_macro.dropna()

## Data Summary: 2-Node AVX Master

In [32]:
df_n2_macro_summary = df_n2_macro.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
df_n2_macro_summary 

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,2,26,2,1,1,1,52,125
1,2,27,2,1,1,1,54,125
2,2,28,2,1,1,1,56,125
3,2,29,2,1,1,1,58,125
4,2,30,2,1,1,1,60,125
5,2,31,2,1,1,1,62,125
6,2,32,2,1,1,1,64,125
7,4,13,4,1,1,1,52,61
8,4,14,4,1,1,1,56,61
9,4,15,4,1,1,1,60,61


## 2-Node Macro Cases

In [33]:
# Creating differnt data frames
dfn2macro_2mpi_32omp = df_n2_macro[(df_n2_macro.omp_thrd == 32) & (df_n2_macro.GLB_T/df_n2_macro.LOC_T == 2) & (df_n2_macro.GLB_X/df_n2_macro.LOC_X == 1) & (df_n2_macro.GLB_Y/df_n2_macro.LOC_Y ==1)& (df_n2_macro.GLB_Z/df_n2_macro.LOC_Z == 1)]
dfn2macro_4mpi_16omp = df_n2_macro[(df_n2_macro.omp_thrd == 16) & (df_n2_macro.GLB_T/df_n2_macro.LOC_T == 4) & (df_n2_macro.GLB_X/df_n2_macro.LOC_X == 1) & (df_n2_macro.GLB_Y/df_n2_macro.LOC_Y ==1)& (df_n2_macro.GLB_Z/df_n2_macro.LOC_Z == 1)]
dfn2macro_8mpi_8omp = df_n2_macro[(df_n2_macro.omp_thrd == 8) & (df_n2_macro.GLB_T/df_n2_macro.LOC_T == 8) & (df_n2_macro.GLB_X/df_n2_macro.LOC_X == 1) & (df_n2_macro.GLB_Y/df_n2_macro.LOC_Y ==1)& (df_n2_macro.GLB_Z/df_n2_macro.LOC_Z ==1)]
dfn2macro_16mpi_4omp = df_n2_macro[(df_n2_macro.omp_thrd == 4) & (df_n2_macro.GLB_T/df_n2_macro.LOC_T == 16) & (df_n2_macro.GLB_X/df_n2_macro.LOC_X == 1) & (df_n2_macro.GLB_Y/df_n2_macro.LOC_Y ==1)& (df_n2_macro.GLB_Z/df_n2_macro.LOC_Z == 1)]
dfn2macro_32mpi_2omp = df_n2_macro[(df_n2_macro.omp_thrd == 2) & (df_n2_macro.GLB_T/df_n2_macro.LOC_T == 32) & (df_n2_macro.GLB_X/df_n2_macro.LOC_X == 1) & (df_n2_macro.GLB_Y/df_n2_macro.LOC_Y ==1)& (df_n2_macro.GLB_Z/df_n2_macro.LOC_Z == 1)]
dfn2macro_64mpi_1omp = df_n2_macro[(df_n2_macro.omp_thrd == 1) & (df_n2_macro.GLB_T/df_n2_macro.LOC_T == 64) & (df_n2_macro.GLB_X/df_n2_macro.LOC_X == 1) & (df_n2_macro.GLB_Y/df_n2_macro.LOC_Y ==1)& (df_n2_macro.GLB_Z/df_n2_macro.LOC_Z == 1)]


### 2-Node Macro FLOPs Measuement

In [34]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2macro_2mpi_32omp.Dirc_Data_Mov*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2macro_2mpi_32omp.Fused_GFLOPS*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_32omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn2macro_2mpi_32omp.Dirc_Data_Mov*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2macro_2mpi_32omp.Dphi_GFLOPS*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_32omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2macro_4mpi_16omp.Dirc_Data_Mov*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                        y=(dfn2macro_4mpi_16omp.Fused_GFLOPS*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_16omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2macro_4mpi_16omp.Dirc_Data_Mov*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                         y=(dfn2macro_4mpi_16omp.Dphi_GFLOPS*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_16omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn2macro_8mpi_8omp.Dirc_Data_Mov*(dfn2macro_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2macro_8mpi_8omp.Fused_GFLOPS*(dfn2macro_8mpi_8omp.MPI_size/2)),
                         mode='lines+markers', 
                         name='Fused_8mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn2macro_8mpi_8omp.Dirc_Data_Mov*(dfn2macro_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2macro_8mpi_8omp.Dphi_GFLOPS*(dfn2macro_8mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_8mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn2macro_16mpi_4omp.Dirc_Data_Mov*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2macro_16mpi_4omp.Fused_GFLOPS*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_4omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2macro_16mpi_4omp.Dirc_Data_Mov*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2macro_16mpi_4omp.Dphi_GFLOPS*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_4omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn2macro_32mpi_2omp.Dirc_Data_Mov*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2macro_32mpi_2omp.Fused_GFLOPS*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_2omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn2macro_32mpi_2omp.Dirc_Data_Mov*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2macro_32mpi_2omp.Dphi_GFLOPS*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_2omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))



fig.add_trace(go.Scatter(x=(dfn2macro_64mpi_1omp.Dirc_Data_Mov*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2macro_64mpi_1omp.Fused_GFLOPS*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_64mpi_1omp',
                         line=dict(color='olive',width=3)))
fig.add_trace(go.Scatter(x=(dfn2macro_64mpi_1omp.Dirc_Data_Mov*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2macro_64mpi_1omp.Dphi_GFLOPS*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_64mpi_1omp',
                         line=dict(color='lightskyblue',width=3)))


# Adding a vertical line
fig.add_vline(x=80e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=140, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='80e6'
    , y=130
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"2_Node_Macro" GFlops Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

In [35]:
# dfn2macro_2mpi_32omp
# dfn2macro_4mpi_16omp
# dfn2macro_8mpi_8omp
# dfn2macro_16mpi_4omp
# dfn2macro_32mpi_2omp
# dfn2macro_64mpi_1omp

### 2-Node Macro FLOPs Measuement

In [36]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2macro_2mpi_32omp.Dirc_Data_Mov*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2macro_2mpi_32omp.Fused_BAND*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_2mpi_32omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))
fig.add_trace(go.Scatter(x=(dfn2macro_2mpi_32omp.Dirc_Data_Mov*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         y=(dfn2macro_2mpi_32omp.Dphi_BAND*(dfn2macro_2mpi_32omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_2mpi_32omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2macro_4mpi_16omp.Dirc_Data_Mov*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                        y=(dfn2macro_4mpi_16omp.Fused_BAND*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_4mpi_16omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2macro_4mpi_16omp.Dirc_Data_Mov*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                         y=(dfn2macro_4mpi_16omp.Dphi_BAND*(dfn2macro_4mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_4mpi_16omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))


fig.add_trace(go.Scatter(x=(dfn2macro_8mpi_8omp.Dirc_Data_Mov*(dfn2macro_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2macro_8mpi_8omp.Dphi_BAND*(dfn2macro_8mpi_8omp.MPI_size/2)),
                         mode='lines+markers', 
                         name='Fused_8mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))
fig.add_trace(go.Scatter(x=(dfn2macro_8mpi_8omp.Dirc_Data_Mov*(dfn2macro_8mpi_8omp.MPI_size/2)), 
                         y=(dfn2macro_8mpi_8omp.Dphi_BAND*(dfn2macro_8mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_8mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


fig.add_trace(go.Scatter(x=(dfn2macro_16mpi_4omp.Dirc_Data_Mov*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2macro_16mpi_4omp.Fused_BAND*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_16mpi_4omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))
fig.add_trace(go.Scatter(x=(dfn2macro_16mpi_4omp.Dirc_Data_Mov*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         y=(dfn2macro_16mpi_4omp.Dphi_BAND*(dfn2macro_16mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_16mpi_4omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=7,line=dict(color='MediumPurple',width=2))))


fig.add_trace(go.Scatter(x=(dfn2macro_32mpi_2omp.Dirc_Data_Mov*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2macro_32mpi_2omp.Fused_BAND*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_32mpi_2omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))
fig.add_trace(go.Scatter(x=(dfn2macro_32mpi_2omp.Dirc_Data_Mov*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         y=(dfn2macro_32mpi_2omp.Dphi_BAND*(dfn2macro_32mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_32mpi_2omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=7, line=dict(color='mediumvioletred',width=1))))



fig.add_trace(go.Scatter(x=(dfn2macro_64mpi_1omp.Dirc_Data_Mov*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2macro_64mpi_1omp.Fused_BAND*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Fused_64mpi_1omp',
                         line=dict(color='olive',width=3)))
fig.add_trace(go.Scatter(x=(dfn2macro_64mpi_1omp.Dirc_Data_Mov*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         y=(dfn2macro_64mpi_1omp.Dphi_BAND*(dfn2macro_64mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='Dphi_64mpi_1omp',
                         line=dict(color='lightskyblue',width=3)))

# Adding vertical and horizontal lines 
fig.add_vline(x=80e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=153, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='80e6'
    , y=310
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=155
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"2_Node_Macro" Bandwidth Measurement: on Different Computational Resources',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside")) 


In [37]:
# dfn2macro_2mpi_32omp
# dfn2macro_4mpi_16omp
# dfn2macro_8mpi_8omp
# dfn2macro_16mpi_4omp
# dfn2macro_32mpi_2omp
# dfn2macro_64mpi_1omp